# Item 19 : Provide Optional Behavior with Key-word Arguments

Like most other programming languages, calling a function in Python allows for passing arguments by position.

In [1]:
def remainder(number, divisor):
    return number % divisor

In [2]:
assert remainder(20, 7) == 6

In [4]:
remainder(20, 7)
remainder(20, divisor=7)
remainder(number=20, divisor=7)
remainder(divisor=7, number=20)

6

In [5]:
remainder(number=20, 7)

SyntaxError: positional argument follows keyword argument (<ipython-input-5-9265fd4030d2>, line 1)

In [6]:
def flow_rate(weight_diff, time_diff):
    return weight_diff / time_diff

In [8]:
weight_diff = 0.5
time_diff = 3
flow = flow_rate(weight_diff, time_diff)

In [10]:
print('%.3f kg per second' % flow)

0.167 kg per second


In [11]:
def flow_Rate(weight_diff, time_diff, period=1, units_per_kg=1):
    return ((weight_diff / units_per_kg) / time_diff) * period

In [16]:
def my_func(*args):
    print(args)
    

def my_generator():
    for i in range(10):
        yield i

In [15]:
my_func(1, 2, 3)

(1, 2, 3)


In [24]:
for i in my_generator():
    print(i)

0
1
2
3
4
5
6
7
8
9


In [33]:
it = my_generator()

In [35]:
my_func(*it)

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


# Item 20 : Use None and Docstrings to Specify Dynamic Default Arguments

Sometimes you need to use a non-static type as a keyword argument's default value. For example, say you want to print logging messages taht are marked with the time of the logged event. In the default case, you want the message to include the time when the function was called. You might try the following approach, assuming the default arguments are reevaluated each time the function is called.

In [1]:
from datetime import datetime

def log(message, when=datetime.now()):
    print('%s:%s' % (when, message))

In [2]:
from time import sleep

log('Hi there!')
sleep(0.1)
log('Hi again!')

2018-06-25 08:26:50.399652:Hi there!
2018-06-25 08:26:50.399652:Hi again!


In [3]:
def log(message, when=None):
    when = datetime.now() if when is None else when
    print('%s:%s' % (when, message))

In [4]:
log('Hi there!')
sleep(0.1)
log('Hi again!')

2018-06-25 08:29:20.115693:Hi there!
2018-06-25 08:29:20.217037:Hi again!


In [5]:
import json

def decode(date, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default

In [6]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1

NameError: name 'data' is not defined

In [7]:
def decode(data, default=None):
    if default is None:
        default = {}
    try:
        return json.loads(data)
    except ValueError:
        return default

# Item 21 : Enforce Clarity with Keyword-Only Arguments

Passing arguments by keyword is a powerful feature of Python functions. The flexibility of keyword arguments enables you to write code that will be clear for your use cases. For example, say you want to divide one number by another but 

In [2]:
def safe_division(number, divisor, ignore_overflow,
                 ignore_zero_division):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

Using this function is straightforward. This call will ignore the float overflow from division and will return zero.

In [3]:
result = safe_division(1, 10**500, True, False)
print(result)

0.0


In [5]:
result = safe_division(1, 0, False, True)
print(result)

inf


The problem is that it's easy to confuse the postion of the two Boolean arguments that control the exception-ignoring behavior. This can easily cause bugs that are hard to track down. One way to improve the readability of this code is to use keyword arguments. By default, the function can b overly cautious and can always re-raise exceptions.

In [6]:
def safe_division_b(number, divisor,
                   ignore_overflow=False,
                   ignore_zero_division=False):
    pass

With complex functions like this, it's better to require that callers are clear abou their intentions. In Python 3, you can demand clarity by defining your functions with keyword-only arguments. These arguments can only be supplied by keyword, never by position. Here, I redefine the safe_division function to accept keyword-only arguments.The * symbol in the argument list indicates the end of positional arguments and the begining of keyword-only arguments.

In [8]:
def safe_division_c(number, divisor, *, ignore_overflow=False, ignore_zero_division=False):
    pass

### Keyword-Only Arguments in Python 2

In [9]:
# Python 2
def print_args(*args, **kwargs):
    print 'Positional:', args
    print 'Keyword: ', kwargs

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(int 'Positional:', args)? (<ipython-input-9-8c8e46eef19f>, line 3)

In [10]:
def safe_division_d(number, divisor, **kwargs):
    ignore_overflow = kwargs.pop('ignore_overflow', False)
    ignore_zero_div = kwargs.pop('ignore_zero_division', False)
    if kwargs:
        raise TypeError('Unexpected **kwargs: %r' % kwargs)
    pass

# Item 22 : Prefer Helper Classes Over Bookkeeping with Dictionaries and Tuples

In [11]:
class SimpleGradebook(object):
    def __init__(self):
        self._grades = {}
    
    def add_student(self, name):
        self._grades[name] = []
        
    def report_grade(self, name, score):
        self._grades[name].append(score)

    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

Python's built-in dictionary and tuple types made it easy to keep going, adding later after to the internal bookkeeping. But you should avoid doing this for more than one level of nesting. It makes your code hard to read by other programmers and sets you up for a maintenance nightmare.
 As soon as you realize the bookkeeping is getting comlicated, break it all out into classes. This lets you provide well-defined interfaces that better encapsulate your data. This also enables you to create a layer of abstraction between your interfaces and your concrete implementations.

## Refactoring to Classes

You can start moving to classes at the bottom of the dependency tree: a single grade. A class seems too heavyweight for such simple infomation. A tuple, though, seems appropriate because grades are immutable. Here, I use the uple (score, weight)
to track grades in a list:

In [12]:
import collections
Grade = collections.namedtuple('Grade', ('score', 'weight'))

In [13]:
type(Grade)

type

In [15]:
Grade.score

In [16]:
class Subject(object):
    def __init__(self):
        self._grades = []
        
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
        
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

In [17]:
class Student(object):
    def __init__(self):
        self._subjects = {}
        
    def subject(self, name):
        if name not in self._subjects:
            self._subjects[name] = Subjects()
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count
    

class Gradebook(object):
    def __init__(self):
        self._students = {}
        
    def students(self, name):
        if name not in self._students:
            self._students[name] = Students()
        return self._students[name]

# Item 23 : Accept Functions for Simple Interfaces Instead of Classes

Many of Python's built-in APIs allow you to customize behavior by passing in a function. These hooks are used by APIs to call back your code while they execute. For example, the list type's sort method takes an optional key argument that's used to determine each index's value for sorting. Here, I sort a list of names based on their lengths by providing a lambda expression as the key hook:

In [18]:
def log_missing():
    print('Key added')
    return 0

In [19]:
current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]
result = defaultdict(log_missing, current)
print('Before:', dict(result))
for key, amount in increments:
    result[key] += amount
print('After:', dict(result))

NameError: name 'defaultdict' is not defined